# Guía Data Science - Conectando Pandas con SQL


IMPORTANTE: Te recomendamos correr este notebook en google colab para que no tengas que instalar nada en tu computador. Puedes subir el archivo comunas.db que utilizaremos más adelante, en la sección de Archivos (hay un ícono de carpeta en el menú del lado izquierdo). Si no se sube este archivo, la base de datos `comunas.db` queda sin tablas, y el código no funcionará correctamente.

## Utilizando datos externos y SQL

Para analizar datos y aprender como trabajar con SQL, usaremos la base de datos `comunas.db` que usa el siguiente esquema: 

````
Comunas(cod INT PRIMARY KEY, 
        nombre VARCHAR(200), 
        provincia VARCHAR(200), 
        region VARCHAR(200), 
        superficie FLOAT, 
        poblacion INT, 
        densidad FLOAT, 
        idh FLOAT)
````


Primero importaremos algunas librerías necesarias como `sqlite3` y `pandas` y luego realizaremos la conexión con la base de datos.

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('comunas.db')
conn.text_factory = lambda x: str(x, 'utf-8')
curr = conn.cursor()

Ahora que hemos realizado la conexión,  usaremos una consulta SQL para extraer los datos y guardarlos en un dataframe. Comúnmente la idea es preparar los datos con SQL y extraer a pandas solo lo necesario.

In [3]:
curr.execute("SELECT * FROM Comunas;")
com_frame = pd.DataFrame(curr.fetchall())
com_frame

,0,1,2,3,4,5,6,7
0,15101,Arica,Arica,Arica y Parinacota,4799.4,210936,736.0,38.40
1,15102,Camarones,Arica,Arica y Parinacota,3927.0,679,0.3,751.00
2,15201,Putre,Parinacota,Arica y Parinacota,5902.5,1462,0.2,707.00
3,15202,General Lagos,Parinacota,Arica y Parinacota,2244.4,739,0.5,0.67
4,1101,Iquique,Iquique,Tarapacá,2242.1,184953,82.4,766.00
...,...,...,...,...,...,...,...,...
341,13601,Talagante,Talagante,Metropolitana de Santiago,126.0,78887,474.6,749.00
342,13602,El Monte,Talagante,Metropolitana de Santiago,118.0,32468,224.2,688.00
343,13603,Isla de Maipo,Talagante,Metropolitana de Santiago,189.0,33723,136.5,724.00
344,13604,Padre Hurtado,Talagante,Metropolitana de Santiago,80.8,50696,626.2,728.00


Renombremos los nombres de las columnas del dataframe según nuestro esquema:

In [4]:
com_frame.rename(columns = { 0: 'cod', 1: 'nombre', 2: 'provincia', 3: 'region', 
                            4: 'superficie', 5: 'poblacion', 6: 'densidad', 7: 'idh'}, inplace=True)

Usemos `head` y `describe` para tener una mejor idea de como se ven nuestros datos:

In [5]:
com_frame.head(10)

,cod,nombre,provincia,region,superficie,poblacion,densidad,idh
0,15101,Arica,Arica,Arica y Parinacota,4799.4,210936,736.0,38.40
1,15102,Camarones,Arica,Arica y Parinacota,3927.0,679,0.3,751.00
2,15201,Putre,Parinacota,Arica y Parinacota,5902.5,1462,0.2,707.00
3,15202,General Lagos,Parinacota,Arica y Parinacota,2244.4,739,0.5,0.67
4,1101,Iquique,Iquique,Tarapacá,2242.1,184953,82.4,766.00
5,1107,Alto Hospicio,Iquique,Tarapacá,572.9,94455,87.6,NaN
6,1401,Pozo Almonte,Tamarugal,Tarapacá,13765.8,11519,0.7,722.00
7,1402,Camiña,Tamarugal,Tarapacá,2200.2,1156,0.5,619.00
8,1403,Colchane,Tamarugal,Tarapacá,4015.6,1384,0.4,603.00
9,1404,Huara,Tamarugal,Tarapacá,10474.6,2360,0.2,676.00


In [6]:
com_frame.describe()

,cod,superficie,poblacion,densidad,idh
count,346.000000,3.460000e+02,346.000000,346.000000,341.000000
mean,8563.156069,5.815967e+03,50407.078035,884.646821,637.527654
std,3356.540795,6.727407e+04,86926.943488,2668.330303,188.322684
min,1101.000000,7.000000e+00,127.000000,0.000000,0.510000
25%,6109.250000,2.513000e+02,9148.500000,9.325000,637.000000
50%,8313.500000,6.331500e+02,17688.000000,26.100000,672.000000
75%,11176.250000,1.657500e+03,50747.000000,89.575000,719.000000
max,15202.000000,1.250258e+06,805000.000000,15667.000000,949.000000


# Eliminando valores nulos

En esta tabla tenemos valores nulos. Vamos a buscarlos. Primero vamos a encontrar todas las filas que contengan algún nulo, para luego filtrar por ese arreglo.

In [7]:
com_frame.isnull().any(axis=1)

0      False
1      False
2      False
3      False
4      False
       ...  
341    False
342    False
343    False
344    False
345    False
Length: 346, dtype: bool

In [8]:
com_frame[com_frame.isnull().any(axis=1)]

,cod,nombre,provincia,region,superficie,poblacion,densidad,idh
5,1107,Alto Hospicio,Iquique,Tarapacá,572.9,94455,87.6,NaN
177,8112,Hualpén,Concepción,Biobío,53.5,86176,1610.8,NaN
198,8314,Alto Biobío,Biobío,Biobío,2124.6,10039,4.7,NaN
219,9121,Cholchol,Cautín,La Araucanía,427.9,10825,25.3,NaN
288,12202,Antártica,Antártica Chilena,Magallanes y Antártica Chilena,1250257.6,127,0.0,NaN


`pandas` tiene métodos auxiliares para lidiar con datos faltantes. Uno es eliminar aquellas filas con la función `dropna()`

In [9]:
com_cleaned = com_frame.dropna()
com_cleaned

,cod,nombre,provincia,region,superficie,poblacion,densidad,idh
0,15101,Arica,Arica,Arica y Parinacota,4799.4,210936,736.0,38.40
1,15102,Camarones,Arica,Arica y Parinacota,3927.0,679,0.3,751.00
2,15201,Putre,Parinacota,Arica y Parinacota,5902.5,1462,0.2,707.00
3,15202,General Lagos,Parinacota,Arica y Parinacota,2244.4,739,0.5,0.67
4,1101,Iquique,Iquique,Tarapacá,2242.1,184953,82.4,766.00
...,...,...,...,...,...,...,...,...
341,13601,Talagante,Talagante,Metropolitana de Santiago,126.0,78887,474.6,749.00
342,13602,El Monte,Talagante,Metropolitana de Santiago,118.0,32468,224.2,688.00
343,13603,Isla de Maipo,Talagante,Metropolitana de Santiago,189.0,33723,136.5,724.00
344,13604,Padre Hurtado,Talagante,Metropolitana de Santiago,80.8,50696,626.2,728.00


O podemos tomar una opción menos radical, que es reemplazar los nulos por un valor en particular.

In [10]:
com_frame = com_frame.fillna(0)
com_frame

,cod,nombre,provincia,region,superficie,poblacion,densidad,idh
0,15101,Arica,Arica,Arica y Parinacota,4799.4,210936,736.0,38.40
1,15102,Camarones,Arica,Arica y Parinacota,3927.0,679,0.3,751.00
2,15201,Putre,Parinacota,Arica y Parinacota,5902.5,1462,0.2,707.00
3,15202,General Lagos,Parinacota,Arica y Parinacota,2244.4,739,0.5,0.67
4,1101,Iquique,Iquique,Tarapacá,2242.1,184953,82.4,766.00
...,...,...,...,...,...,...,...,...
341,13601,Talagante,Talagante,Metropolitana de Santiago,126.0,78887,474.6,749.00
342,13602,El Monte,Talagante,Metropolitana de Santiago,118.0,32468,224.2,688.00
343,13603,Isla de Maipo,Talagante,Metropolitana de Santiago,189.0,33723,136.5,724.00
344,13604,Padre Hurtado,Talagante,Metropolitana de Santiago,80.8,50696,626.2,728.00


# Agregación en Pandas

Para obtener la cantidad de habitantes por región haremos lo siguiente:

In [11]:
com_frame['poblacion'].groupby(com_frame['region']).sum() # Ojo! esto retorna un objeto Series

region
Antofagasta                             551627
Arica y Parinacota                      213816
Atacama                                 292054
Aysén del Gral. C. Ibáñez del Campo     106893
Biobío                                 1565839
Coquimbo                                714856
La Araucanía                            933537
Lib. Gral. Bernardo O'Higgins           903248
Los Lagos                               835829
Los Ríos                                380618
Magallanes y Antártica Chilena          158828
Maule                                  1073635
Metropolitana de Santiago              7090580
Tarapacá                                300021
Valparaíso                             1859312
Ñuble                                   460156
Name: poblacion, dtype: int64

Podemos preguntar cuántos elementos hay por grupo. En este caso obtendríamos el número de comunas por región.

In [12]:
com_frame['poblacion'].groupby(com_frame['region']).size() # Ojo! esto retorna un objeto Series

region
Antofagasta                             9
Arica y Parinacota                      4
Atacama                                 9
Aysén del Gral. C. Ibáñez del Campo    10
Biobío                                 33
Coquimbo                               15
La Araucanía                           32
Lib. Gral. Bernardo O'Higgins          33
Los Lagos                              30
Los Ríos                               12
Magallanes y Antártica Chilena         11
Maule                                  30
Metropolitana de Santiago              52
Tarapacá                                7
Valparaíso                             38
Ñuble                                  21
Name: poblacion, dtype: int64

En `pandas` se pueden hacer operaciones mucho más complejas, pero no veremos nada avanzado en esta ocasión. Puedes revisar la documentación para ver que más puedes hacer.

In [15]:
com_frame['superficie'].groupby(com_frame['region']).max()

region
Antofagasta                              30718.1
Arica y Parinacota                        5902.5
Atacama                                  18664.0
Aysén del Gral. C. Ibáñez del Campo      29796.4
Biobío                                    2124.6
Coquimbo                                  7610.0
La Araucanía                              3914.2
Lib. Gral. Bernardo O'Higgins             2597.0
Los Lagos                                 8470.5
Los Ríos                                  3292.0
Magallanes y Antártica Chilena         1250257.6
Maule                                     4504.0
Metropolitana de Santiago                 4994.8
Tarapacá                                 13765.8
Valparaíso                                1455.0
Ñuble                                     1776.6
Name: superficie, dtype: float64